In [ ]:
from flask import Flask, request, jsonify
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from flask_cors import CORS
from pyngrok import ngrok

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer.src_lang = "eng_Latn" 
target_lang = "tel_Telu"


def translate(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=64)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_lang), 
        max_length=64
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
app = Flask(_name_)
CORS(app)

@app.route('/process', methods=['POST'])
def process_request():
    data = request.get_json()
    sentence = data.get("sentence", "")
    
    if not sentence:
        return jsonify({"error": "No sentence provided"}), 400
    
    result = translate(sentence)
    return jsonify({"processed_sentence": result})

public_url = ngrok.connect(5003)
print(f"🔗 Ngrok tunnel running at: {public_url}")

app.run(port=5003, debug=False, use_reloader=False)